In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# Wave 2020

In [ ]:
# Load the first wave of the survey
data = pd.read_csv(
    '../data/raw/scalar/SCALAR_Coastal_Longitudinal_Study_Wave_One_NL.csv')

## Missing and duplicate values


In [ ]:
# Print number of columns and rows
print(f"Number of columns: {data.shape[1]}")
print(f"Number of rows: {data.shape[0]}")

# Print number of columns that have missing values
print(f"Number of columns with missing values: {data.isnull().any().sum()}")

# Print number of rows that have missing values
print(f"Number of rows with missing values: {data.isnull().any(axis=1).sum()}")\

# Print column names that have missing values
print(f"Columns with missing values: {data.columns[data.isnull().any()]}")

# Print % of missing values in the columns with missing values
cols_with_missing = data.columns[data.isnull().any()]
for col in cols_with_missing:
    print(f"{col}: {data[col].isnull().mean()*100:.2f}%")

In [ ]:
# Print number of rows and unique respondents
print(f"Number of rows: {data.shape[0]}")
print(f"Number of unique respondents: {data['ID'].nunique()}")

## Mapping


In [ ]:
# Define response mappings for all variables
response_mappings = {
    # Home type
    'Q1_home_ID_NL_US': {
    1: 'Apartment',
    2: 'Semidetached house or townhouse',
    3: 'Independent house',
    4: 'Mobile home',
    97: 'Other'
    },

    # Resilience
    'R01_resilience_5': {
        1: 'Strongly agree',
        2: 'Somewhat agree',
        3: 'Neither agree nor disagree',
        4: 'Somewhat disagree',
        5: 'Strongly disagree'
    },

    'R01_resilience_6': {
        1: 'Strongly agree',
        2: 'Somewhat agree',
        3: 'Neither agree nor disagree',
        4: 'Somewhat disagree',
        5: 'Strongly disagree'
    },


    # Responsibility
    'Q15_responsibility': {
        1: "Completely government",
        2: "Mostly government", 
        3: "Equal responsibility",
        4: "Mostly individual",
        5: "Completely individual",
    },

    # Government measures
    'Q22_gov_measures': {
        1: "Sufficient long-term",
        2: "Sufficient short-term",
        3: "Not sufficient",
        98: "Don't know"
    },

    # Social expectation
    'Q44_social_expectation': {
        1: "No expectation",
        2: "Low expectation",
        3: "Moderate expectation",
        4: "High expectation",
        5: "Strong expectation",
    },

    # Community member
    'Q10_community': {
        0: "No",
        1: "Yes"
    },

    # Adaptation others
    'R07_adaptation_others': {
        1: "None of them",
        2: "One",
        3: "Two",
        4: "Three",
        5: "Four",
        6: "Five",
        7: "More than five",
        98: "Don't know"
    },

    # Flood risk perceptions
    "R02_perc_prob": {
        1: "My house is completely safe",
        2: "Less often than 1 in 500 years", 
        3: "Once in 500 years",
        4: "Once in 200 years",
        5: "Once in 100 years",
        6: "Once in 50 years",
        7: "Once in 10 years",
        8: "Annually",
        9: "More frequent than once per year",
        98: "Don't know"
    },
    "Q24_perc_prob_change": {
        1: "Increase",
        2: "Stay the same", 
        3: "Decrease",
        98: "Don't know"
    },
    "R03_perc_damage": {
        1: "Not at all severe",
        2: "Slightly severe",
        3: "Moderately severe", 
        4: "Quite severe",
        5: "Very severe",
        98: "Don't know/Prefer not to say"
    },
    "R04_perc_health": {
        1: "Not at all likely",
        2: "Hardly likely",
        3: "Somewhat likely",
        4: "Quite likely", 
        5: "Almost certain"
    },
    "R05_worry": {
        1: "Not at all worried",
        2: "A little worried",
        3: "Somewhat worried",
        4: "Quite worried",
        5: "Very worried"
    },

    # Flood experience
    "Q18_flood_exp": {
        0: "No",
        1: "Yes"
    },
    "Q18a_flood_where": {
        1: "At home",
        2: "During a trip",
        3: "Both at home and during a trip"
    },
    "Q18c_flood_health": {
        0: "No",
        1: "Yes",
        96: "Not applicable",
        99: "Prefer not to say"
    },
    "Q18ci_flood_work": {
        1: "One month",
        2: "Two months", 
        3: "Three months",
        4: "Four months",
        5: "Five months",
        6: "Six months",
        7: "More than six months",
        98: "Don't know / Can't remember"
    },

    # Housing & Accommodation
    "Q1_home_ID_NL_US": {
        1: "Apartment",
        2: "Semidetached house or townhouse",
        3: "Independent house", 
        4: "Mobile home",
        97: "Other"
    },
    "Q2_floors_1": {
        0: "No",
        1: "Yes"
    },
    "Q2_floors_2": {
        0: "No", 
        1: "Yes"
    },
    "Q2_floors_3": {
        0: "No",
        1: "Yes"
    },
    "Q3_constr_qual": {
        1: "Very high-quality construction",
        2: "High-quality construction",
        3: "Medium-quality construction",
        4: "Old building with many issues"
    },
    "Q4_home_size_CN_ID_NL": {
        1: "Less than 50 square metres",
        2: "Between 50 and 75 square metres",
        3: "Between 76 and 100 square metres",
        4: "Between 101 and 125 square metres", 
        5: "Between 125 and 150 square metres",
        6: "More than 151 square metres",
        98: "Don't know"
    },
    "Q5_home_tenure": {
        1: "Rent",
        2: "Own",
        97: "Other"
    },
    "Q5a_mortgage": {
        1: "0-20%",
        2: "21-40%",
        3: "41-60%",
        4: "61-80%",
        5: "81-99%",
        6: "Fully paid off",
        98: "Don't know/prefer not to say"
    },
    "Q8_move_out": {
        1: "1 year or less",
        2: "1-5 years",
        3: "5-10 years", 
        4: "More than 10 years",
        98: "Don't know"
    },
    "Q9_easy_leave": {
        1: "Very difficult",
        2: "Difficult",
        3: "Moderately difficult",
        4: "Easy",
        5: "Very easy"
    },

    # Housing Choice Factors
    "Q11_search_improve": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_social": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_family": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_area": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_job": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_location": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_hazard": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_other": {
        0: "No",
        1: "Yes"
    },
    "Q11_search_dontknow": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type1": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type2": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type3": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type4": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type5": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type6": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type7": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type8": {
        0: "No",
        1: "Yes"
    },
    "Q11a_hazard_type9": {
        0: "No",
        1: "Yes"
    },
    "Q11a_not_say": {
        0: "No",
        1: "Yes"
    },

    # Location Relative to Dikes
    "Q46_dikes_NL": {
        0: "Outside dike ring",
        1: "Inside dike ring",
        98: "Don't know"
    },

    # Aggregated implementation responses
    "R2_implementation_SM1_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM2_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM3_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM4_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM5_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM6_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },
    "R2_implementation_SM7_agg": {
        1: "Already implemented",
        2: "Will implement",
        3: "Will implement",
        4: "Will implement",
        5: "Will implement",
        6: "Will not implement"
    },

     # Disaggregated implementation responses
    "R2_implementation_SM1": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM2": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM3": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM4": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM5": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM6": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },
    "R2_implementation_SM7": {
        1: "Already implemented",
        2: "Next 6 months",
        3: "Next 12 months",
        4: "Next 2 years",
        5: "After 2 years",
        6: "Will not implement"
    },

    # Socio-demographics
    "Q0_age": {
        1: "16-24",
        2: "25-34",
        3: "35-44",
        4: "45-54",
        5: "55-64",
        6: "65+"
    },
    "Q0_gender": {
        1: "Male",
        2: "Female"
    },
    "Q0_education_NL": {
        1: "Less than secondary education",
        2: "Secondary education",
        3: "College/tertiary education",
        4: "Postgraduate"
    },
    "Q0_employment_ID_NL": {
        1: "Working full time",
        2: "Working part time",
        3: "Full time student",
        4: "Retired",
        5: "Unemployed",
        6: "Not working",
        7: "Full-time home-maker or housewife",
        97: "Other"
    },
    "Q57_hh_size_NL": {
        1: "1",
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8 or more",
        98: "Don't know",
        99: "Prefer not to say"
    },
    "Q59_disability": {
        0: "No",
        1: "Yes",
        99: "Prefer not to say"
    },
    "Q60_child": {
        0: "No",
        1: "Yes"
    },
    "Q60_elder": {
        0: "No",
        1: "Yes"
    },
    "Q60_no": {
        0: "No",
        1: "Yes"
    },
    "Q60_not_say": {
        0: "No",
        1: "Yes"
    },
    "Q61_single_parent": {
        0: "No",
        1: "Yes",
        99: "Prefer not to say"
    },

    # Income & Savings
    "Q52_multiple_income": {
        0: "No",
        1: "Yes"
    },
    "Q52a_income_job": {
        1: "1-10%",
        2: "11-20%",
        3: "21-30%",
        4: "31-40%",
        5: "41-50%",
        6: "51-60%",
        7: "61-70%",
        8: "71-80%",
        9: "81-90%",
        10: "91-100%",
        99: "Prefer not to say"
    },
    "Q53_income_NL": {
        1: "Less than 26130 Euro",
        2: "Between 26131 and 42785 Euro",
        3: "Between 42786 and 66935 Euro",
        4: "Between 66936 and 102540 Euro",
        5: "More than 102540 Euro",
        99: "Prefer not to say"
    },
    "R08_economic_comfort": {
        1: "Very difficult to live",
        2: "Difficult to live",
        3: "Coping",
        4: "Living comfortably",
        5: "Living very comfortably",
        99: "Prefer not to say"
    },
    "R09_savings_change": {
        1: "Less savings than 2 years ago",
        2: "Same savings as 2 years ago",
        3: "More savings than 2 years ago",
        4: "No applicable, since no savings",
        98: "Don't know",
        99: "Prefer not to say"
    },
    "Q56_savings_change_future": {
        1: "Less savings than 2 years ago",
        2: "Same savings as 2 years ago",
        3: "More savings than 2 years ago",
        4: "No savings",
        98: "Don't know",
        99: "Prefer not to say"
    },
    "Q58_savings": {
        1: "Little to no savings",
        2: "Half month's wages",
        3: "1 month's wages",
        4: "1.5 month's wages",
        5: "2 month's wages",
        6: "3 month's wages",
        7: "4+ month's wages",
        98: "Don't know",
        99: "Prefer not to say"
    },

    # Employment & Industry
    "Q47b_industry_type_NL": {
        1: "Transport Retail or Wholesale",
        2: "Business",
        3: "Manufacturing or Agriculture",
        4: "Hospitality, Catering, or Tourism",
        5: "Health or Social Care",
        6: "Public Sector, Government or Education",
        7: "Construction",
        8: "Homemaker",
        97: "Other sector"
    },
    "Q48_business_owner": {
        0: "No",
        1: "Yes"
    },
    "Q49_self_employed_NL": {
        0: "No",
        1: "Yes"
    },
    "Q50_employer_size": {
        1: "1 (just me)",
        2: "2",
        3: "3 to 5",
        4: "6 to 9",
        5: "10 to 19",
        6: "20 to 34",
        7: "35 to 49",
        8: "50 to 99",
        9: "100 to 249",
        10: "250 to 499",
        11: "500 to 999",
        12: "1000 or more",
        96: "Not applicable",
        98: "Don't know"
    },
    "Q51_unempl_time": {
        1: "Less than 1 month",
        2: "1 to 3 months",
        3: "3 to 6 months",
        4: "More than 7 months"
    },

    # Community & Social Ties
    "Q10_community": {
        0: "No",
        1: "Yes"
    }
}

# Define column mappings
column_mappings = {
    'R01_resilience_5': 'rely_on_fam_friends',
    'R01_resilience_6': 'rely_on_gov',

    # Flood risk perceptions
    "R02_perc_prob": "perceived_flood_frequency",
    "Q24_perc_prob_change": "perceived_risk_change_10y",
    "Q27_perc_prob_30y": "perceived_flood_probability_30y",
    "R03_perc_damage": "perceived_damage_severity",
    "R04_perc_health": "perceived_health_risk",
    "R05_worry": "flood_worry_level",

    # Flood experience
    "Q18_flood_exp": "experienced_flood",
    "Q18a_flood_where": "flood_location",
    "Q18b_flood_year": "flood_year",
    "Q18c_flood_health": "health_impact",
    "Q18ci_flood_work": "work_months_missed",
    "Q18d_flood_cost": "financial_loss",

    # Self efficacy 
    'R1a_self_efficacy_SM1': 'se1_raise_ground_floor_level',
    'R1a_self_efficacy_SM2': 'se2_strengthen_foundations',
    'R1a_self_efficacy_SM3': 'se3_reinforce_walls_floor',
    'R1a_self_efficacy_SM4': 'se4_raise_electricity_meter',
    'R1a_self_efficacy_SM5': 'se5_install_anti_backflow_valves',
    'R1a_self_efficacy_SM6': 'se6_install_pump_drainage',
    'R1a_self_efficacy_SM7': 'se7_fix_water_barriers',

    # Response efficacy
    'R1b_resp_efficacy_SM1': 're1_raise_ground_floor_level',
    'R1b_resp_efficacy_SM2': 're2_strengthen_foundations',
    'R1b_resp_efficacy_SM3': 're3_reinforce_walls_floor',
    'R1b_resp_efficacy_SM4': 're4_raise_electricity_meter',
    'R1b_resp_efficacy_SM5': 're5_install_anti_backflow_valves',
    'R1b_resp_efficacy_SM6': 're6_install_pump_drainage',
    'R1b_resp_efficacy_SM7': 're7_fix_water_barriers',

    # Perceived costs
    'R1c_perc_cost_SM1': 'pc1_raise_ground_floor_level',
    'R1c_perc_cost_SM2': 'pc2_strengthen_foundations',
    'R1c_perc_cost_SM3': 'pc3_reinforce_walls_floor',
    'R1c_perc_cost_SM4': 'pc4_raise_electricity_meter',
    'R1c_perc_cost_SM5': 'pc5_install_anti_backflow_valves',
    'R1c_perc_cost_SM6': 'pc6_install_pump_drainage',
    'R1c_perc_cost_SM7': 'pc7_fix_water_barriers',

    # Damage reduction
    'Q37_dam_reduction': 'damage_reduction',

    # 
    'R2_implementation_SM1': 'adapt1_raise_ground_floor_level',
    'R2_implementation_SM2': 'adapt2_strengthen_foundations',
    'R2_implementation_SM3': 'adapt3_reinforce_walls_floor',
    'R2_implementation_SM4': 'adapt4_raise_electricity_meter',
    'R2_implementation_SM5': 'adapt5_install_anti_backflow_valves',
    'R2_implementation_SM6': 'adapt6_install_pump_drainage',
    'R2_implementation_SM7': 'adapt7_fix_water_barriers',

    'R2_implementation_SM1_agg': 'adapt1_raise_ground_floor_level_agg',
    'R2_implementation_SM2_agg': 'adapt2_strengthen_foundations_agg',
    'R2_implementation_SM3_agg': 'adapt3_reinforce_walls_floor_agg',
    'R2_implementation_SM4_agg': 'adapt4_raise_electricity_meter_agg',
    'R2_implementation_SM5_agg': 'adapt5_install_anti_backflow_valves_agg',
    'R2_implementation_SM6_agg': 'adapt6_install_pump_drainage_agg',
    'R2_implementation_SM7_agg': 'adapt7_fix_water_barriers_agg',

    # Housing & Accommodation
    "Q1_home_ID_NL_US": "home_type",
    "Q2_floors_1": "has_basement",
    "Q2_floors_2": "has_ground_floor",
    "Q2_floors_3": "has_upper_floor",
    "Q3_constr_qual": "construction_quality",
    "Q4_home_size_CN_ID_NL": "home_size",
    "Q5_home_tenure": "tenure_type",
    "Q5a_mortgage": "mortgage_paid_off",
    "Q5b_home_sell": "estimated_home_value",
    "Q6_home_costs": "monthly_housing_costs",
    "Q7_move_in": "move_in_year",
    "Q8_move_out": "planned_stay_duration",
    "Q9_easy_leave": "ease_of_leaving",

    # Housing Choice Factors
    "Q11_search_improve": "moved_for_improvement",
    "Q11_search_social": "moved_for_social",
    "Q11_search_family": "moved_for_family",
    "Q11_search_area": "moved_for_area",
    "Q11_search_job": "moved_for_job",
    "Q11_search_location": "moved_for_location",
    "Q11_search_hazard": "moved_for_hazards",
    "Q11_search_other": "moved_other_reason",
    "Q11_search_dontknow": "moved_reason_unknown",
    "Q11a_hazard_type1": "moved_from_hurricanes",
    "Q11a_hazard_type2": "moved_from_storms",
    "Q11a_hazard_type3": "moved_from_rainfall",
    "Q11a_hazard_type4": "moved_from_flooding",
    "Q11a_hazard_type5": "moved_from_sealevel",
    "Q11a_hazard_type6": "moved_from_fires",
    "Q11a_hazard_type7": "moved_from_earthquakes",
    "Q11a_hazard_type8": "moved_from_tsunamis",
    "Q11a_hazard_type9": "moved_from_temperature",
    "Q11a_not_say": "moved_hazard_not_say",

    # Location Relative to Dikes
    "Q46_dikes_NL": "dike_location",

    # Socio-demographics
    "Q0_age": "age",
    "Q0_gender": "gender",
    "Q0_postcode": "postcode",
    "Q0_education_NL": "education",
    "Q0_employment_ID_NL": "employment",

    # Household composition & vulnerable members
    "Q57_hh_size_NL": "household_size",
    "Q59_disability": "disability",
    "Q60_child": "child",
    "Q60_elder": "elder",
    "Q60_no": "no_children_or_elders",
    "Q60_not_say": "prefer_not_to_say",
    "Q61_single_parent": "single_parent",

    # Income & Savings
    "Q52_multiple_income": "multiple_income",
    "Q52a_income_job": "income_job_pct",
    "Q53_income_NL": "income_level",
    "R08_economic_comfort": "economic_comfort",
    "R09_savings_change": "savings_change",
    "R09a_savings_change_frac": "savings_change_frac",
    "Q56_savings_change_future": "savings_change_future",
    "Q56a_savings_change_future_frac": "savings_change_future_frac",
    "Q58_savings": "savings_amount",

    # Employment & Industry
    "Q47b_industry_type_NL": "industry_type",
    "Q48_business_owner": "business_owner",
    "Q49_self_employed_NL": "self_employed",
    "Q50_employer_size": "employer_size",
    "Q51_unempl_time": "unemployment_time",

    # Community & Social Ties
    "Q10_community": "community_member",
    
    # Postal code
    'Q0_postcode': "Q0_postcode",

    # Responsibility
    'Q15_responsibility': 'responsibility_perception',
    'Q22_gov_measures': 'gov_measures_perception',
    'Q44_social_expectation': 'social_expectation',
    'R07_adaptation_others': 'adaptation_others',

    # 
    'Q1_home_ID_NL_US': 'home_type',
    'Q2_floors_1': 'use_basement',
    'Q2_floors_2': 'use_ground_floor',
    'Q2_floors_3': 'use_first_floor_and_above'
}

df = data.copy()
df = df.set_index("ID")

# Get all columns to process
X_cols = [col for col in column_mappings.keys(
) if not col.startswith("R2_implementation")]
y_cols = [col for col in column_mappings.keys(
) if col.startswith("R2_implementation") and not col.endswith("_agg")]

r2_cols = [col for col in column_mappings.keys() if col.startswith("R2_implementation") and not col.endswith("_agg")]

# Form a new column for each R2_implementation column
for col in r2_cols:
    df[col + '_agg'] = df[col].replace(response_mappings[col + '_agg'])

r2_cols_agg = [col + '_agg' for col in r2_cols]

# Create copy of data with selected columns
X = df[X_cols].copy()
y = df[y_cols].copy()

# Apply response mappings where they exist
for col in X_cols:
    if col in response_mappings:
        X[col] = X[col].replace(response_mappings[col])

for col in y_cols:
    if col in response_mappings:
        y[col] = y[col].replace(response_mappings[col])

# Rename columns
X = X.rename(columns=column_mappings)
y = y.rename(columns=column_mappings)

categories = ["Already implemented", "Next 6 months", "Next 12 months", "Next 2 years", "After 2 years", "Will not implement"]
y = y.apply(lambda x: pd.Categorical(x, categories=categories, ordered=True))

agg_categories = ["Already implemented", "Will implement", "Will not implement"]
y_agg = df[r2_cols_agg].copy()
y_agg = y_agg.apply(lambda x: pd.Categorical(x, categories=agg_categories, ordered=True))
y_agg.rename(columns=column_mappings, inplace=True)

## Binning

In [ ]:
# Define bins and labels for numeric columns
bins = {
    'perceived_flood_probability_30y': {
        'bins': [0, 20, 40, 60, 80, 100],
        'labels': ['0-20%', '21-40%', '41-60%', '61-80%', '81-100%']
    },
    'flood_year': {
        'bins': [0, 1989, 2004, 2014, 2020], 
        'labels': ['Historical (pre-1990)', '1990-2004', '2005-2014', '2015-2020']
    },
    'financial_loss': {
        'bins': [0, 10, 1000, 2000, 10000, 200000],
        'labels': ['Minimal (≤€10)', 'Small (€11-€1k)', 'Medium (€1k-€2k)', 
                   'Large (€2k-€10k)', 'Very Large (>€10k)']
    }
}

# Function to bin numeric columns
def bin_column(series, bins_dict):
    # Get mask of non-NA values in original series
    non_na_mask = ~series.isna()
    
    binned = pd.cut(series, 
                    bins=bins_dict['bins'],
                    labels=bins_dict['labels'],
                    include_lowest=True)
    
    # Only check for NAs in values that weren't NA to begin with
    if binned[non_na_mask].isna().any():
        raise ValueError(f"Some non-NA values could not be binned for column {series.name}")
    return binned

# Apply binning to numeric columns in X where applicable
for col, bins_dict in bins.items():
    if col in X.columns:
        X[f'{col}_binned'] = bin_column(X[col], bins_dict)
        
        # Print distribution statistics
        print(f"\nDistribution of binned {col}:")
        print(X[f'{col}_binned'].value_counts().sort_index())
        print("\nPercentages:")
        print(round(X[f'{col}_binned'].value_counts(normalize=True).sort_index() * 100, 2))

In [ ]:
# Aggregate household size
household_size_map = {
    'Prefer not to say': 'Prefer not to say',
    "Don't know": "Don't know",
    '1': '1',
    '2': '2',
    '3': '3-5',
    '4': '3-5',
    '5': '5+',
    '6': '5+',
    '7': '5+',
    '8 or more': '5+'
}
X['household_size_agg'] = X['household_size'].map(household_size_map)

## Extra questions

In [ ]:
# community_member
# responsibility_perception
# gov_measures_perception
# social_expectation
# dike_location
# adaptation_others
# tenure_type
extra_questions = ['responsibility_perception',
                   'gov_measures_perception',
                   'community_member', 
                   'social_expectation',
                   'adaptation_others',
                   'dike_location',  
                   'tenure_type',
                   'home_type',
                   'use_basement',
                   'use_ground_floor',
                   'use_first_floor_and_above'
                   ]
for q in extra_questions:
    # print(data[q].value_counts(dropna=False))
    print(q, X[q].unique())
    print('-'*100)

## Merge and save the result

In [ ]:
result = pd.merge(X, y, left_index=True, right_index=True)
result = pd.merge(result, y_agg, left_index=True, right_index=True)

# Create a directory for the processed data if it doesn't exist
os.makedirs('../data/processed/scalar', exist_ok=True)

# Save the result
result.to_csv('../data/processed/scalar/wave_1.csv', index=True)

# Wave 2023

In [ ]:
# Load the first wave of the survey
data = pd.read_csv(
    '../data/raw/scalar/SCALAR_Coastal_Study_new_respondents_Wave_Five_NL.csv')

## Reconstructing the variable

In [ ]:
# Reconstruct responsibility perception variable
cols_of_interest = [
    'Q35_responsibility_citizens',
    'Q35_responsibility_business',
    'Q35_responsibility_ngo_local',
    'Q35_responsibility_ngo_international',
    'Q35_responsibility_gov_local',
    'Q35_responsibility_gov_national',
    'Q35_responsibility_w_authorities',
    'Q35_responsibility_other'
]
df = data[cols_of_interest].copy().apply(pd.to_numeric)

for col in cols_of_interest:
    # //10 gives 0 for 0–9, 1 for 10–19, …, 10 for 100
    # *10 brings it back to 0,10,20,…,100
    df[col + "_bin"] = (df[col] // 10) * 10

df_binned = df.drop(columns=cols_of_interest)

# Sum up all gov‐type columns
gov_cols = [
    "Q35_responsibility_gov_local_bin",
    "Q35_responsibility_gov_national_bin",
    "Q35_responsibility_w_authorities_bin",
]

ind_cols = ['Q35_responsibility_citizens_bin',
            'Q35_responsibility_business_bin']

df["gov_share"] = df[gov_cols].sum(axis=1)
df["ind_share"] = df[ind_cols].sum(axis=1)

def classify(row):
    g, i = row["gov_share"], row["ind_share"]

    if g == 100:
        return "Completely government"
    elif i == 100:
        return "Completely individual"
    elif g > 50 and g < 100:
        return "Mostly government"
    elif i > 50 and i < 100:
        return "Mostly individual"
    else:
        # e.g. g ≤ 50 and i ≤ 50  →  responsibility is shared
        return "Equal responsibility"

df["responsibility_perception"] = df.apply(classify, axis=1)

# Finally, tally up
summary = df["responsibility_perception"].value_counts().rename_axis(
    "responsibility_perception").reset_index(name="count")
print(summary)


In [ ]:
# Get value counts and convert to percentages
counts = df['responsibility_perception'].value_counts()
percentages = counts / len(df) * 100

# Create bar plot
ax = percentages.plot(kind='bar')

# Add percentage labels on top of each bar
for i, v in enumerate(percentages):
    ax.text(i, v, f'{v:.1f}%', ha='center', va='bottom')

# Improve readability
plt.xticks(rotation=45, ha='right')
plt.ylabel('Percent of Responses (%)')
plt.xlabel('')

# Remove all spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Add newly constructed variables to the data
data['responsibility_perception'] = df['responsibility_perception']

## Missing and duplicate variables

In [ ]:
# Print number of columns and rows
print(f"Number of columns: {data.shape[1]}")
print(f"Number of rows: {data.shape[0]}")

# Print number of columns that have missing values
print(f"Number of columns with missing values: {data.isnull().any().sum()}")

# Print number of rows that have missing values
print(f"Number of rows with missing values: {data.isnull().any(axis=1).sum()}")\

# Print column names that have missing values
print(f"Columns with missing values: {data.columns[data.isnull().any()]}")

# Print % of missing values in the columns with missing values
cols_with_missing = data.columns[data.isnull().any()]
for col in cols_with_missing:
    print(f"{col}: {data[col].isnull().mean()*100:.2f}%")

## Binning

In [ ]:
data['Q0_age'].unique()
age_bins = [16, 24, 34, 44, 54, 64, 100]
data['age_binned'] = pd.cut(data['Q0_age'], bins=age_bins, labels=['16-24', '25-34', '35-44', '45-54', '55-64', '65+'])
data['age_binned'].value_counts(dropna=False)

In [ ]:
# Mapping detailed education levels to aggregated categories
education_level_agg_map = {
    1: "Less than secondary education",
    3: "Less than secondary education",
    4: "Secondary education",
    5: "Secondary education",
    6: "College/tertiary education",
    7: "College/tertiary education",
    8: "College/tertiary education",
    9: "Postgraduate",
    10: "Postgraduate"
}

data['education_level_agg'] = data['Q0_education_NL'].map(education_level_agg_map)

gender_agg_map = {1: 'Male',
                  2: 'Female',
                  3: 'Female',
                  4: 'Male',
                  5: 'Prefer not to say',
                  99: 'Prefer not to say'}

data['gender_agg'] = data['Q0_gender'].map(gender_agg_map)

## Mapping

In [ ]:
response_mappings = {
    'age_binned': {
        "16-24": "16-24",
        "25-34": "25-34",
        "35-44": "35-44",
        "45-54": "45-54",
        "55-64": "55-64",
        "65+": "65+"
    },
     'Q58_savings': {
         1: "Little to no savings",
         2: "Half month's wages",
         3: "1 month's wages",
         4: "1.5 month's wages",
         5: "2 month's wages",
         6: "3 month's wages",
         7: "4+ month's wages",
         98: "Don't know",
         99: "Prefer not to say"
     },
    
    'gender_agg': {
        'Male': 'Male',
        'Female': 'Female',
        'Prefer not to say': 'Prefer not to say'
    },

    'responsibility_perception': {
        'Completely government': 'Completely government',
        'Mostly government': 'Mostly government',
        'Equal responsibility': 'Equal responsibility',
        'Mostly individual': 'Mostly individual',
        'Completely individual': 'Completely individual'
    },

    'education_level_agg': {
        'Less than secondary education': 'Less than secondary education',
        'Secondary education': 'Secondary education',
        'College/tertiary education': 'College/tertiary education',
        'Postgraduate': 'Postgraduate'
    },

    'Q53_income_NL': {
        1: 'Less than 26130 Euro',
        2: 'Between 26131 and 42785 Euro',
        3: 'Between 42786 and 66935 Euro',
        4: 'Between 66936 and 102540 Euro',
        5: 'More than 102540 Euro',
        99: 'Prefer not to say'
    },

    # Resilience
    'R01_resilience_5': {
        1: 'Strongly agree',
        2: 'Somewhat agree',
        3: 'Neither agree nor disagree',
        4: 'Somewhat disagree',
        5: 'Strongly disagree'
    },

    'R01_resilience_6': {
        1: 'Strongly agree',
        2: 'Somewhat agree',
        3: 'Neither agree nor disagree',
        4: 'Somewhat disagree',
        5: 'Strongly disagree'
    },

    'R02_perc_prob': {
        1: 'My house is completely safe',
        2: 'Less often than 1 in 500 years',
        3: 'Once in 500 years or a 0.2% chance annually',
        4: 'Once in 200 years or a 0.5% chance annually',
        5: 'Once in 100 years or 1% chance annually',
        6: 'Once in 50 years or a 2% chance annually',  
        7: 'Once in 10 years or 10% chance annually',
        8: 'Annually',
        9: 'More frequent than once per year',
        97: 'Other',
        98: 'Don\'t know'
    },
    'R05_worry': {
        1: 'Not at all worried',
        2: 'A little worried',
        3: 'Somewhat worried',
        4: 'Quite worried',
        5: 'Very worried',
        98: 'Don\'t know'
    },
    'Q18_flood_exp': {
        0: 'No',
        1: 'Yes'
    },
    
    # Default adaptive behavior responses
    "R2_implementation_SM1": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },

    "R2_implementation_SM2": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM3": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM4": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM5": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM6": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM7": {
        1: 'Already implemented',
        2: 'In the near future (next 1-3 years)',
        3: 'At some point in future (next 3-5 years)',
        4: 'Will not implement'
    },
    "R2_implementation_SM1_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM2_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM3_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM4_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM5_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM6_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
    "R2_implementation_SM7_agg": {
        1: 'Already implemented',
        2: 'Will implement',
        3: 'Will implement',
        4: 'Will not implement'
    },
}

column_mappings = {
    'age_binned': 'age_binned',
    'Q58_savings': 'savings_amount',
    'Q53_income_NL': 'income_level',
    'gender_agg': 'gender_agg',
    'education_level_agg': 'education_level_agg',
    'responsibility_perception': 'responsibility_perception',

    # Resilience
    'R01_resilience_5': 'rely_on_fam_friends',
    'R01_resilience_6': 'rely_on_gov',
    
    # Threat appraisal
    'R02_perc_prob': 'perceived_flood_frequency',
    'R05_worry': 'flood_worry_level',
    'Q18_flood_exp': 'experienced_flood',
    
    # Coping appraisal
    # Self efficacy 
    'R1a_self_efficacy_SM1': 'se1_raise_ground_floor_level',
    'R1a_self_efficacy_SM2': 'se2_strengthen_foundations',
    'R1a_self_efficacy_SM3': 'se3_reinforce_walls_floor',
    'R1a_self_efficacy_SM4': 'se4_raise_electricity_meter',
    'R1a_self_efficacy_SM5': 'se5_install_anti_backflow_valves',
    'R1a_self_efficacy_SM6': 'se6_install_pump_drainage',
    'R1a_self_efficacy_SM7': 'se7_fix_water_barriers',

    # Response efficacy
    'R1b_resp_efficacy_SM1': 're1_raise_ground_floor_level',
    'R1b_resp_efficacy_SM2': 're2_strengthen_foundations',
    'R1b_resp_efficacy_SM3': 're3_reinforce_walls_floor',
    'R1b_resp_efficacy_SM4': 're4_raise_electricity_meter',
    'R1b_resp_efficacy_SM5': 're5_install_anti_backflow_valves',
    'R1b_resp_efficacy_SM6': 're6_install_pump_drainage',
    'R1b_resp_efficacy_SM7': 're7_fix_water_barriers',

    # Perceived costs
    'R1c_perc_cost_SM1': 'pc1_raise_ground_floor_level',
    'R1c_perc_cost_SM2': 'pc2_strengthen_foundations',
    'R1c_perc_cost_SM3': 'pc3_reinforce_walls_floor',
    'R1c_perc_cost_SM4': 'pc4_raise_electricity_meter',
    'R1c_perc_cost_SM5': 'pc5_install_anti_backflow_valves',
    'R1c_perc_cost_SM6': 'pc6_install_pump_drainage',
    'R1c_perc_cost_SM7': 'pc7_fix_water_barriers',

     # Aadaptive behavior 
     # Default one
    'R2_implementation_SM1': 'adapt1_raise_ground_floor_level',
    'R2_implementation_SM2': 'adapt2_strengthen_foundations',
    'R2_implementation_SM3': 'adapt3_reinforce_walls_floor',
    'R2_implementation_SM4': 'adapt4_raise_electricity_meter',
    'R2_implementation_SM5': 'adapt5_install_anti_backflow_valves',
    'R2_implementation_SM6': 'adapt6_install_pump_drainage',
    'R2_implementation_SM7': 'adapt7_fix_water_barriers',

    # Aggregated one
    'R2_implementation_SM1_agg': 'adapt1_raise_ground_floor_level_agg',
    'R2_implementation_SM2_agg': 'adapt2_strengthen_foundations_agg',
    'R2_implementation_SM3_agg': 'adapt3_reinforce_walls_floor_agg',
    'R2_implementation_SM4_agg': 'adapt4_raise_electricity_meter_agg',
    'R2_implementation_SM5_agg': 'adapt5_install_anti_backflow_valves_agg',
    'R2_implementation_SM6_agg': 'adapt6_install_pump_drainage_agg',
    'R2_implementation_SM7_agg': 'adapt7_fix_water_barriers_agg',
}

df = data.copy()
df = df.set_index("ID")

# Get all columns to process
X_cols = [col for col in column_mappings.keys(
) if not col.startswith("R2_implementation")]
y_cols = [col for col in column_mappings.keys(
) if col.startswith("R2_implementation") and not col.endswith("_agg")]

r2_cols = [col for col in column_mappings.keys() if col.startswith("R2_implementation") and not col.endswith("_agg")]

# Form a new column for each R2_implementation column
for col in r2_cols:
    df[col + '_agg'] = df[col].replace(response_mappings[col + '_agg'])

r2_cols_agg = [col + '_agg' for col in r2_cols]

# Create copy of data with selected columns
X = df[X_cols].copy()
y = df[y_cols].copy()

# Apply response mappings where they exist
for col in X_cols:
    if col in response_mappings:
        X[col] = X[col].replace(response_mappings[col])

for col in y_cols:
    if col in response_mappings:
        y[col] = y[col].replace(response_mappings[col])

# Rename columns
X = X.rename(columns=column_mappings)
y = y.rename(columns=column_mappings)

categories = ["Already implemented", 'In the near future (next 1-3 years)', 'At some point in future (next 3-5 years)', 'Will not implement']
y = y.apply(lambda x: pd.Categorical(x, categories=categories, ordered=True))

agg_categories = ["Already implemented", "Will implement", "Will not implement"]
y_agg = df[r2_cols_agg].copy()
y_agg = y_agg.apply(lambda x: pd.Categorical(x, categories=agg_categories, ordered=True))
y_agg.rename(columns=column_mappings, inplace=True)

## Merge and save result

In [ ]:
result = pd.merge(X, y, left_index=True, right_index=True)
result = pd.merge(result, y_agg, left_index=True, right_index=True)

# Create a directory for the processed data if it doesn't exist
os.makedirs('../data/processed/scalar', exist_ok=True)

# Save the result
result.to_csv('../data/processed/scalar/wave_5.csv', index=True)